In [9]:
from Autolabeler import workout
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import warnings
import numpy as np
from pycaret.classification import *

# Load Test Data

In [15]:
folder_path = "LABELED"
os.makedirs(folder_path, exist_ok=True)
file_names = []
for name in os.listdir(folder_path):
    file_path = f"{folder_path}/{name}"
    file_names.append(file_path)
#Ahora tengo que especificar mis features 
signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
data = workout(filelist=file_names, signals= signals)
df= pd.concat(data)

In [16]:
df.shape

(164840, 31)

In [17]:
X = df.drop(['peaks', 'first_sample','last_sample', 'first_sample_closest_peak', 
             'last_sample_closest_peak','exercise','exercising_periods'], axis=1)
X.columns

Index(['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', 'magnX', 'magnY',
       'magnZ', 'linAccX', 'linAccY', 'linAccZ', 'accX_mod', 'accY_mod',
       'accZ_mod', 'gyroX_mod', 'gyroY_mod', 'gyroZ_mod', 'magnX_mod',
       'magnY_mod', 'magnZ_mod', 'linAccX_mod', 'linAccY_mod', 'linAccZ_mod'],
      dtype='object')

# Auxiliary Functions

In [26]:
def prepare_data(chunk_df):
    signals = ["accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ", "magnX", "magnY", "magnZ", "linAccX", "linAccY", "linAccZ"]
    stats_dict = {}
    series = chunk_df[signals]
    for column in series.columns:
        mean = series[column].mean()
        std = series[column].std()
        median = series[column].median()
    
        stats_dict[f"{column}_mean"] = mean
        stats_dict[f"{column}_std"] = std
        stats_dict[f"{column}_median"] = median
        
    return pd.DataFrame.from_dict(stats_dict, orient="index").T

# Load Model

In [19]:
# Load the model
pipeline = load_model(model_name="squats_traditional")

Transformation Pipeline and Model Successfully Loaded


In [43]:
ventana = prepare_data(X.iloc[0:170])
prediction = predict_model(pipeline, ventana, raw_score=True)
prediction.prediction_label.iloc[0]

'NO_EXERCISE'

# Check Data

In [45]:
tamano_ventana = 170 
nuevo_df = pd.DataFrame()
nuevo_df_predicciones = pd.DataFrame()

for i in range(0, len(X) - tamano_ventana + 1, tamano_ventana):
    warnings.filterwarnings("ignore")
    ventana = X.iloc[i:i + tamano_ventana]
    
    X1 = prepare_data(ventana)
    prediction = predict_model(pipeline, X1, raw_score=True)
    y_pred = prediction.prediction_label.iloc[0]
    X1['start'] = ventana.index[0]
    X1['end'] = ventana.index[-1]
    X1['prediccion'] = y_pred
    nuevo_df_predicciones = pd.concat([nuevo_df_predicciones, X1], ignore_index=True)

In [52]:
nuevo_df_predicciones.shape

(969, 39)

In [46]:
nuevo_df_predicciones.head()

,accX_mean,accX_std,accX_median,accY_mean,accY_std,accY_median,accZ_mean,accZ_std,accZ_median,gyroX_mean,...,linAccX_median,linAccY_mean,linAccY_std,linAccY_median,linAccZ_mean,linAccZ_std,linAccZ_median,start,end,prediccion
0,0.320889,0.132123,0.346972,8.982878,0.890013,9.229446,2.880478,0.665896,3.129650,9.945353,...,-0.048269,0.099805,0.187570,0.067922,-0.360106,1.004781,-0.052835,1970-01-01 00:00:00.000,1970-01-01 00:00:01.606,NO_EXERCISE
1,-0.603787,0.766760,-0.407991,8.726922,3.044928,9.448398,-3.940197,4.444279,-2.378824,-4.148943,...,0.022855,0.065328,0.563701,0.046574,0.293226,4.306579,-0.221224,1970-01-01 00:00:01.615,1970-01-01 00:00:03.222,NO_EXERCISE
2,-0.403951,0.774089,0.014357,9.344208,2.560344,9.590775,-2.425908,4.935146,-0.017024,27.023705,...,0.063224,0.068588,0.370559,0.070765,0.689761,3.742911,-0.047036,1970-01-01 00:00:03.231,1970-01-01 00:00:04.838,NO_EXERCISE
3,-0.109088,0.393120,0.077770,8.793008,1.531081,9.516596,0.143442,1.987607,1.202163,-27.714648,...,0.021129,0.037734,0.175710,0.023867,-0.792098,1.539848,-0.119698,1970-01-01 00:00:04.847,1970-01-01 00:00:06.454,NO_EXERCISE
4,-0.647070,0.808834,-0.490546,8.755257,3.869784,9.309609,-2.912006,4.438672,-1.028028,-1.784178,...,0.043280,0.080496,0.402918,0.078759,-0.035546,4.949006,-0.259682,1970-01-01 00:00:06.467,1970-01-01 00:00:08.074,NO_EXERCISE


In [47]:
np.unique(nuevo_df_predicciones.prediccion,return_counts=True)

(array(['NO_EXERCISE', 'SQUAT'], dtype=object), array([853, 116], dtype=int64))

In [50]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == 'SQUAT'][['start', 'end', 'prediccion']]
squats_detection

,start,end,prediccion
42,1970-01-01 00:00:05.971,1970-01-01 00:00:07.578,SQUAT
55,1970-01-01 00:00:26.990,1970-01-01 00:00:28.597,SQUAT
66,1970-01-01 00:00:44.778,1970-01-01 00:00:46.385,SQUAT
68,1970-01-01 00:00:48.014,1970-01-01 00:00:49.621,SQUAT
69,1970-01-01 00:00:49.630,1970-01-01 00:00:51.237,SQUAT
...,...,...,...
953,1970-01-01 00:00:35.291,1970-01-01 00:00:36.880,SQUAT
957,1970-01-01 00:00:41.684,1970-01-01 00:00:43.272,SQUAT
960,1970-01-01 00:00:46.479,1970-01-01 00:00:48.068,SQUAT
964,1970-01-01 00:00:52.872,1970-01-01 00:00:54.460,SQUAT


In [51]:
squats_detection = nuevo_df_predicciones[nuevo_df_predicciones.prediccion == ('NO_EXERCISE'
                                                                              ''
                                                                              '')][['start', 'end', 'prediccion']]
squats_detection

,start,end,prediccion
0,1970-01-01 00:00:00.000,1970-01-01 00:00:01.606,NO_EXERCISE
1,1970-01-01 00:00:01.615,1970-01-01 00:00:03.222,NO_EXERCISE
2,1970-01-01 00:00:03.231,1970-01-01 00:00:04.838,NO_EXERCISE
3,1970-01-01 00:00:04.847,1970-01-01 00:00:06.454,NO_EXERCISE
4,1970-01-01 00:00:06.467,1970-01-01 00:00:08.074,NO_EXERCISE
...,...,...,...
962,1970-01-01 00:00:49.674,1970-01-01 00:00:51.262,NO_EXERCISE
963,1970-01-01 00:00:51.271,1970-01-01 00:00:52.860,NO_EXERCISE
965,1970-01-01 00:00:54.469,1970-01-01 00:00:56.057,NO_EXERCISE
966,1970-01-01 00:00:56.066,1970-01-01 00:00:57.655,NO_EXERCISE
